In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
#%load_ext cuml.accel
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [3]:
df=pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
df[['CabinDeck', 'CabinNum', 'CabinSide']] = df['Cabin'].str.split('/', expand=True)
df['Lastname'] = df["Name"].str.split(' ').str[1]

df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,CabinDeck,CabinNum,CabinSide,Lastname
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,B,0,P,Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,F,0,S,Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,A,0,S,Susent
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,A,0,S,Susent
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,F,1,S,Santantines
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,A,98,P,Noxnuther
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,G,1499,S,Mondalley
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,G,1500,S,Connon
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,E,608,S,Hontichre


In [5]:
X= df.drop(columns=["PassengerId","Name"])
y=df["Transported"]

le = LabelEncoder()
y = le.fit_transform(y)

In [6]:
X

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,CabinDeck,CabinNum,CabinSide,Lastname
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,0,P,Ofracculy
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,0,S,Vines
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,0,S,Susent
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,0,S,Susent
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,1,S,Santantines
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,A,98,P,Noxnuther
8689,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False,G,1499,S,Mondalley
8690,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,G,1500,S,Connon
8691,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,E,608,S,Hontichre


In [7]:
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # handle NaNs
    ('encoder', OneHotEncoder(handle_unknown='ignore'))    # encode after imputin
])

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # fill NaNs
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(transformers=[
    ('cat', cat_pipeline, ["HomePlanet","CryoSleep","Destination","VIP",'CabinDeck','CabinSide',"Lastname"]),
    ("num",num_pipeline,["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck",'CabinNum',])
])

In [8]:
# models = {
#     'LogisticRegression': LogisticRegression(max_iter=5000),
#     'RandomForest': RandomForestClassifier(),
#     'SVC': SVC()
# }

In [9]:
# best_model_name = None
# best_score = 0
# best_estimator = None

# for name, model in models.items():
#     pipe = Pipeline([
#         ('preprocessor', preprocessor),
#         ('classifier', model)
#     ])
#     scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
#     print(f'{name} CV Accuracy: {scores.mean():.3f} ± {scores.std():.3f}')

#     if scores.mean() > best_score:
#         best_score = scores.mean()
#         best_model_name = name
#         best_estimator = pipe

# print(f"\n est model: {best_model_name} with accuracy: {best_score:.3f}")

In [10]:
# best_estimator.fit(X, y)

In [11]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score
# from sklearn.pipeline import Pipeline
# import optuna

# def objective_rf(trial):
#     params = {
#         "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
#         "max_depth": trial.suggest_int("max_depth", 4, 64, log=True),
#         "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
#         "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
#         "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
#         "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
#         "random_state": 42,
#         "n_jobs": -1
#     }

#     pipe = Pipeline([
#         ('preprocessor', preprocessor),  # You must define this before running
#         ('classifier', RandomForestClassifier(**params))
#     ])

#     scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
#     return scores.mean()

# study_rf = optuna.create_study(direction="maximize")
# study_rf.optimize(objective_rf, n_trials=20)

# print("Best RandomForest score:", study_rf.best_value)
# print("Best RandomForest params:", study_rf.best_params)

In [12]:
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import cross_val_score
# from catboost import CatBoostClassifier
# import optuna

# def objective_catboost(trial):
#     params = {
#         "iterations": trial.suggest_int("iterations", 100, 1000),
#         "depth": trial.suggest_int("depth", 4, 10),
#         "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
#         "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
#         "border_count": trial.suggest_int("border_count", 32, 255),
#         "random_strength": trial.suggest_float("random_strength", 1e-3, 10.0, log=True),
#         "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
#         "verbose": 0,
#         "random_state": 42
#     }

#     pipe = Pipeline([
#         ('preprocessor', preprocessor),  # must be defined before
#         ('classifier', CatBoostClassifier(**params))
#     ])

#     scores = cross_val_score(pipe, X, y, cv=5, scoring="accuracy")
#     return scores.mean()

# study_catboost = optuna.create_study(direction="maximize")
# study_catboost.optimize(objective_catboost, n_trials=20)

# print("Best CatBoost score:", study_catboost.best_value)
# print("Best CatBoost params:", study_catboost.best_params)

In [13]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import cross_val_score
# import optuna

# def objective_logreg(trial):
#     penalty = trial.suggest_categorical("penalty", ["l1", "l2", "elasticnet", "none"])
#     C = trial.suggest_float("C", 1e-4, 10.0, log=True)
#     solver = trial.suggest_categorical("solver", ["liblinear", "saga"])

#     if penalty == "elasticnet":
#         l1_ratio = trial.suggest_float("l1_ratio", 0.0, 1.0)
#     else:
#         l1_ratio = None

#     # prune invalid combinations
#     if (penalty == "l1" and solver not in ["liblinear", "saga"]) or \
#        (penalty == "elasticnet" and solver != "saga") or \
#        (penalty == "none" and solver == "liblinear"):
#         raise optuna.exceptions.TrialPruned()

#     model = LogisticRegression(
#         penalty=penalty,
#         C=C,
#         solver=solver,
#         l1_ratio=l1_ratio,
#         max_iter=15000,
#         random_state=42
#     )

#     pipe = Pipeline([
#         ('preprocessor', preprocessor),  # Define this beforehand
#         ('classifier', model)
#     ])

#     scores = cross_val_score(pipe, X, y, cv=5, scoring="accuracy")
#     return scores.mean()

# study_logreg = optuna.create_study(direction="maximize")
# study_logreg.optimize(objective_logreg, n_trials=20)

# print("Best LogisticRegression score:", study_logreg.best_value)
# print("Best LogisticRegression params:", study_logreg.best_params)

In [14]:
rf = RandomForestClassifier(
        n_estimators=102,
        max_depth=11,
        min_samples_split=5,
        min_samples_leaf=13,
        max_features=None,
        bootstrap=True,
        random_state=42,
        n_jobs=-1
)
lgb = LGBMClassifier()
cat = CatBoostClassifier(
        iterations=893,
        depth=9,
        learning_rate=0.01315357713525083,
        l2_leaf_reg=1.1542505875209033,
        border_count=79,
        random_strength=4.116468056114066,
        bagging_temperature=0.5806200933270593,
        verbose=0,
        random_state=42
    )
lg = LogisticRegression(
        penalty='l1',
        C=0.14694999864133876,
        solver='liblinear',
        max_iter=5000,
        random_state=42
    )
gb = GradientBoostingClassifier()


voting_clf = VotingClassifier(
    estimators=[
        ('rf', rf),
        ('gb', gb),
        ('lgb',lgb),
        ('lg',lg),
        ('cat',cat)
    ],
    voting='soft'
)

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', voting_clf)
])

scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
print(f'CV Accuracy: {scores.mean():.3f} ± {scores.std():.3f}')

[LightGBM] [Info] Number of positive: 3502, number of negative: 3452
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002114 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1649
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503595 -> initscore=0.014380
[LightGBM] [Info] Start training from score 0.014380
[LightGBM] [Info] Number of positive: 3502, number of negative: 3452
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000605 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1647
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 27
[LightGBM] [Info] [binary:

In [15]:
best_estimator=pipe.fit(X,y)

[LightGBM] [Info] Number of positive: 4378, number of negative: 4315
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000722 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1650
[LightGBM] [Info] Number of data points in the train set: 8693, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503624 -> initscore=0.014495
[LightGBM] [Info] Start training from score 0.014495


+++++++++++++++++++++++++++++++++++++TEST+++++++++++++++++++++++++++++++++++++++  
LogisticRegression CV Accuracy: 0.781 ± 0.012
RandomForest CV Accuracy: 0.774 ± 0.014
SVC CV Accuracy: 0.720 ± 0.015


LogisticRegression CV Accuracy: 0.784 ± 0.006
RandomForest CV Accuracy: 0.789 ± 0.012
SVC CV Accuracy: 0.786 ± 0.013


CV Accuracy: 0.796 ± 0.007

In [16]:
df = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

In [17]:
df[['CabinDeck', 'CabinNum', 'CabinSide']] = df['Cabin'].str.split('/', expand=True)
df['Lastname'] = df["Name"].str.split(' ').str[1]

In [18]:
X= df.drop(columns=["PassengerId","Name"])

In [19]:
X

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,CabinDeck,CabinNum,CabinSide,Lastname
0,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,G,3,S,Carsoning
1,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,F,4,S,Peckers
2,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,C,0,S,Unhearfus
3,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,C,1,S,Caltilter
4,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,F,5,S,Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,G,1496,S,Peter
4273,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,NaN,NaN,NaN,Scheron
4274,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,D,296,P,Pore
4275,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,D,297,P,Conale


In [20]:
predictions = best_estimator.predict(X)

In [21]:
submission = pd.DataFrame({
    "PassengerId": df["PassengerId"],
    "Transported": predictions.astype(bool)
})

submission.to_csv("submission.csv", index=False)
print("Saved submission.csv for Kaggle")

Saved submission.csv for Kaggle
